In [158]:
%matplotlib inline
import quantstats as qs
import yfinance as yf
import panel as pn

import plotly.express as px

pn.extension("plotly")
import pandas as pd

import hvplot.pandas


from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import HRPOpt
# extend pandas functionality with metrics, etc.
qs.extend_pandas()
window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)
title = 'Stock'
ticker = pn.widgets.input.TextInput()
ticker.value = 'AAPL'
portfolio = None
print(type(portfolio))
init = 0
# input ticker and view stock data

def get_df(ticker, window_size):
    df = yf.Ticker(ticker).history(period="5Y")
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
#     global portfolio
#     if ticker is not '' and portfolio is None:
#         portfolio = df[['Date', 'Close']]
#         portfolio.rename(columns={'Close': ticker}, inplace=True)
#     elif ticker is not '':
#         portfolio = pd.concat([portfolio, df['Close']], join='inner', axis='columns')
#         portfolio.rename(columns={'Close': ticker}, inplace=True)
    
    return df.set_index('Date').rolling(window=window_size).mean().reset_index()
def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return df.hvplot.line('Date', 'Close', grid=True, responsive=True, height=150)

def get_dividends(ticker, window_size):
    stock = yf.Ticker(ticker)
    df = stock.dividends
    
    data = df.resample('Y').sum()
    data = data.iloc[-10:]
    data = data.reset_index()
    data['Year'] = data['Date'].dt.year
    return data.hvplot.bar(x = 'Year', y = 'Dividends', rot=90)
# print(data
# can include additional plots/visualizations to add to the stock panel by creating additional get_plot functions similiar to the one above. 

stock0 = pn.Row(
    pn.Column(title, ticker, window),
    pn.bind(get_plot, ticker, window)
)
stock1 = pn.Row(pn.bind(get_dividends, ticker, window))
stock = pn.Column(stock0, stock1)

# stock

<class 'NoneType'>


In [164]:
# create portfolio analysis
title = 'Portfolio Analysis'
sample_port = ['AAPL', 'GME', 'AMC', 'INTC']
tickers = pn.widgets.input.LiteralInput()
tickers.value = sample_port
def get_df_port(tickers, window_size):

    df = yf.Ticker(tickers[0]).history(period="5Y")
    df.dropna()
    df.reset_index(inplace=True)
    df = df[['Date', 'Close']]
    
    df.rename(columns={'Close': tickers[0]}, inplace=True)
    for i in range(1, len(tickers)):
        temp_df = yf.Ticker(tickers[i]).history(period="5Y")
        temp_df.dropna()
        temp_df.reset_index(inplace=True)
        
        df =  pd.concat([df, temp_df['Close']], join='inner', axis='columns')
        df.rename(columns={'Close': tickers[i]}, inplace=True)
        
    return df.set_index('Date').rolling(window=1).mean().reset_index()

def get_portfolio_opt(tickers, window_size):
        portfolio = get_df_port(tickers, 1)
        portfolio.set_index(keys='Date', inplace=True)
        returns = portfolio.pct_change().dropna()
        hrp = HRPOpt(returns)
        return hrp
    


def get_pie_chart_portfolio(tickers):
    hrp = get_portfolio_opt(tickers, window)
    hrp_weights = hrp.optimize()
    df = pd.DataFrame(hrp_weights.values(), index = hrp_weights.keys(), columns = ['weights']).reset_index()
    return pn.pane.Plotly(px.pie(df, values='weights', names='index', title="Ideal Portfolio Composition"))

def get_portfolio_vol_table(tickers):
    hrp = get_portfolio_opt(tickers, window)
    hrp_weights = hrp.optimize()
    performance = hrp.portfolio_performance()
    df = pd.DataFrame(performance, index=['Expected Annual Return', 'Annual Volatility', 'Sharpe Ratio'], columns=['Value'])
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Performance Measure'}, inplace=True)
    return df.hvplot.table(columns=['Performance Measure', 'Value'])



row0 = pn.Row(
    pn.Column('Porfolio Analysis'),
    pn.bind(get_portfolio_vol_table, tickers))



row1 = pn.Row(pn.Column('Portfolio Weights', tickers), pn.bind(get_pie_chart_portfolio, tickers))
column_port = pn.Column(row1, row0)


In [165]:
spy_daily_returns = get_plot('SPY', 20)
qqq_daily_returns = get_plot('QQQ', 20)
dia_daily_returns =  get_plot('DIA', 20)
spy_sharpe = qs.utils.download_returns('SPY')
qqq_sharpe  = qs.utils.download_returns('QQQ')
dia_sharpe  = qs.utils.download_returns('DIA')


# qqq_snap = qs.plots.snapshot(qqq_daily_returns, title='Performance')
# dia_snap = qs.plots.snapshot(dia_daily_returns, title='Performance')
rowsp = pn.Row('S&P 500', spy_daily_returns )
rowshsp = pn.Row(('Sharpe:', spy_sharpe.sharpe()), ('Volatility', spy_sharpe.volatility()))
rowqqq = pn.Row('Nasdaq', qqq_daily_returns)
rowshq = pn.Row(('Sharpe:', qqq_sharpe.sharpe()), ('Volatility', spy_sharpe.volatility()))
rowdia =  pn.Row('DJIA', dia_daily_returns)
rowshdia = pn.Row(('Sharpe:', spy_sharpe.sharpe()), ('Volatility', spy_sharpe.volatility()))
market = pn.Column(rowsp, rowshsp, rowqqq, rowshq, rowdia, rowshdia)



In [172]:
fb = get_plot('FB', 20)
amzn = get_plot('AMZN', 20)
msft = get_plot('MSFT', 20)
aapl = get_plot('AAPL', 20)
goog = get_plot('GOOG', 20)
fb_stat = qs.utils.download_returns('FB')
amzn_stat = qs.utils.download_returns('AMZN')
aapl_stat = qs.utils.download_returns('AAPL')
goog_stat = qs.utils.download_returns('GOOG')
msft_stat = qs.utils.download_returns('MSFT')

row_fb_plot = pn.Row('FB', fb)
row_amzn_plot = pn.Row('AMZN', amzn)
row_msft_plot = pn.Row('MSFT', msft)
row_aapl_plot = pn.Row('AAPL', aapl)
row_goog_plot = pn.Row('GOOG', goog)
row_msft_stat = pn.Row(('Sharpe:', msft_stat.sharpe()), ('Volatility', msft_stat.volatility()))
row_fb_stat = pn.Row(('Sharpe:', fb_stat.sharpe()), ('Volatility', fb_stat.volatility()))
row_amzn_stat = pn.Row(('Sharpe:', amzn_stat.sharpe()), ('Volatility', amzn_stat.volatility()))
row_aapl_stat = pn.Row(('Sharpe:', aapl_stat.sharpe()), ('Volatility', aapl_stat.volatility()))
row_goog_stat = pn.Row(('Sharpe:', goog_stat.sharpe()), ('Volatility', goog_stat.volatility()))
trending = pn.Column(pn.Row(row_aapl_plot, row_amzn_plot), pn.Row(row_aapl_stat, row_amzn_stat), pn.Row(row_msft_plot, row_goog_plot),
                     pn.Row(row_msft_stat, row_goog_stat), pn.Row(row_fb_plot), row_fb_stat)


In [173]:
pn.Tabs(('Stock Analysis', stock), ('Portfolio Analysis', column_port), ('Market Overview', market), ('Top 5 Stocks by MarketCap', trending), dynamic=True)

Tabs(dynamic=True)
    [0] Column
        [0] Row
            [0] Column
                [0] Markdown(str)
                [1] TextInput(value='AAPL')
                [2] IntSlider(end=21, name='Window Size', start=1, value=6)
            [1] ParamFunction(function)
        [1] Row
            [0] ParamFunction(function)
    [1] Column
        [0] Row
            [0] Column
                [0] Markdown(str)
                [1] LiteralInput(value=['AAPL', 'GME', ...])
            [1] ParamFunction(function)
        [1] Row
            [0] Column
                [0] Markdown(str)
            [1] ParamFunction(function)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] HoloViews(Curve)
        [1] Row
            [0] Str(tuple)
            [1] Str(tuple)
        [2] Row
            [0] Markdown(str)
            [1] HoloViews(Curve)
        [3] Row
            [0] Str(tuple)
            [1] Str(tuple)
        [4] Row
            [0] Markdown(str)
            [1] HoloViews(Curve)
        [5] Row
            [0] Str(tuple)
            [1] Str(tuple)
    [3] Column
        [0] Row
            [0] Row
                [0] Markdown(str)
                [1] HoloViews(Curve)
            [1] Row
                [0] Markdown(str)
                [1] HoloViews(Curve)
        [1] Row
            [0] Row
                [0] Str(tuple)
                [1] Str(tuple)
            [1] Row
                [0] Str(tuple)
                [1] Str(tuple)
        [2] Row
            [0] Row
                [0] Markdown(str)
                [1] HoloViews(Curve)
            [1] Row
                [0] Markdown(str)
                [1] HoloViews(Curve)
        [3] Row
            [0] Row
                [0] Str(tuple)
                [1] Str(tuple)
            [1] Row
                [0] Str(tuple)
                [1] Str(tuple)
        [4] Row
            [0] Row
                [0] Markdown(str)
                [1] HoloViews(Curve)
        [5] Row
            [0] Str(tuple)
            [1] Str(tuple)